In [ ]:
%matplotlib widget

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
from pathlib import Path
import nrrd1
from scipy.stats import zscore
from skimage import exposure
from PIL import Image, ImageDraw

In [ ]:
master = Path(r"\\Funes\Shared\experiments\E0038_confocal\v004_ipn_reference\211108_h2b-mcherry_gad1b-gfp\f1")
anatomy_file1 = tiff.imread(list(master.glob("*C1-Experiment*"))[1])
anatomy_file = tiff.imread(list(master.glob("*C2-Experiment*"))[1])

In [ ]:
# creating a mask:
num_planes, nx, ny = np.shape(anatomy_file)
img = Image.new("L", [nx, ny], 0)
x1 = 300
y2 = 650
y1 = 400
x2 = 300
ImageDraw.Draw(img).polygon([(x1, 0), (x1, y1), (0, y2), (0, 700), (x2, 1024), (1024-x2, 1024), (1024, 700), (1024, y2), (1024-x1, y1), (1024-x1, 0)], outline=1, fill=1)
mask = np.array(img)
print(np.shape(mask))

In [ ]:
fig_mask, ax_mask = plt.subplots(1, 2, figsize=(5,5))
fig_anatomy.tight_layout()


tmp_view = np.nanmean(anatomy_file1, axis=0)
ax_mask[0].imshow(tmp_view, cmap="gray_r", vmin=5, vmax=90, extent=[0,100,0,100])

ax_mask[1].imshow(tmp_view, cmap="gray_r", vmin=5, vmax=90)
ax_mask[1].imshow(mask)

#ax_mask.axis('off')


plt.show()

In [ ]:
fig_anatomy, ax_anatomy = plt.subplots(1, 3, figsize=(15,5))
fig_anatomy.tight_layout()
title_list = ['Horizontal', 'Coronal', 'Sagittal']

k = [0, 6, 2]
extent=[100, 30, 100]
for i in range(3):
    tmp_view = np.nanmean(anatomy_file1, axis=i)
    tmp_view = np.rot90(tmp_view, k=k[i])
    ax_anatomy[i].imshow(tmp_view, cmap="gray_r", vmin=5, vmax=90, extent=[0,100,0,extent[i]])
    ax_anatomy[i].set_title(title_list[i])
    ax_anatomy[i].axis('off')
    
    tmp_view = np.nanmean(anatomy_file, axis=i)
    tmp_view = np.rot90(tmp_view, k=k[i])
    tmp_view = np.ma.masked_where(tmp_view < 0, tmp_view)
    ax_anatomy[i].imshow(tmp_view, cmap="Blues", alpha=0.5, vmin=5, vmax=20, extent=[0,100,0,extent[i]])

plt.show()
file_name = 'anatomy_figure_211108_f1.jpg'
fig_anatomy.savefig(str(master / file_name))

In [ ]:
gad1b_norm2 = np.copy(anatomy_file)
for i in range(num_planes):
    tmp_plane = anatomy_file[i]
    gad1b_norm2[i] = exposure.adjust_gamma(tmp_plane, 1.5)
    #gad1b_norm2[i] = tmp_plane

In [ ]:
gad1b_norm = np.zeros_like(gad1b_norm2, dtype=float)
for i in range(num_planes):
    tmp_plane = zscore(gad1b_norm2[i])
    tmp_plane *= mask
    gad1b_norm[i] = tmp_plane #* i/50

In [ ]:
fig_anatomy, ax_anatomy = plt.subplots(1, 2, figsize=(6,5), gridspec_kw={'width_ratios':[4,1]})
fig_anatomy.tight_layout()


tmp_view = np.nanmean(anatomy_file1, axis=0)
ax_anatomy[0].imshow(tmp_view, cmap="gray_r", vmin=5, vmax=80, extent=[0,100,0,100])
ax_anatomy[0].axis('off')

tmp_view = np.nanmean(gad1b_norm, axis=0)
ax_anatomy[0].imshow(tmp_view, cmap="Blues", alpha=0.5, vmin=0, vmax=1., extent=[0,100,0,100])

tmp_view = np.nanmean(anatomy_file1, axis=2).T
ax_anatomy[1].imshow(tmp_view, cmap="gray_r", vmin=5, vmax=80, extent=[0,25,0,100])
ax_anatomy[1].axis('off')

tmp_view = np.nanmean(gad1b_norm, axis=2).T
ax_anatomy[1].imshow(tmp_view, cmap="Blues", alpha=0.5, vmin=0, vmax=1., extent=[0,25,0,100])
fig_anatomy.tight_layout()
plt.show()
file_name = 'anatomy_figure_211108_f1_zscore_masked.jpg'
fig_anatomy.savefig(str(master / file_name))

In [ ]:
files = list(master.glob("*_f*"))
fish_dir = files[1] 
anatomy_file = tiff.imread(fish_dir)
print(fish_dir)

In [ ]:
fig_anatomy, ax_anatomy = plt.subplots(1, 3, figsize=(15,5))
fig_anatomy.tight_layout()
title_list = ['Horizontal', 'Sagittal', 'Coronal']

k = [3, 6, 2]
for i in range(3):
    tmp_view = np.nansum(anatomy_file, axis=i)
    tmp_view = np.rot90(tmp_view, k=k[i])
    ax_anatomy[i].imshow(tmp_view, cmap="gray_r", vmin=50000, vmax=450000)
    ax_anatomy[i].set_title(title_list[i])
    ax_anatomy[i].axis('off')

fig_anatomy.tight_layout() 
plt.show()
file_name = 'anatomy_figure_confocal_211108_f1.jpg'
fig_anatomy.savefig(str(master / file_name))